**Test with Iris dataset and Test and train split function**

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X, y = iris.data, iris.target
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

### Architecture definition

In [ ]:
import torch.nn as nn

class ANN(nn.Module) :
    def __init__(self, num_classes=3):
        super(self.__class__, self).__init__()

        self.fullyConnected = nn.Sequential(
            nn.Linear(in_features=4, out_features=10),
            nn.ReLU(),
            nn.Linear(in_features=10, out_features=20),
            nn.ReLU(),
            nn.Linear(in_features=20, out_features=num_classes),
        )
    
    def forward(self, x, debug=False):
        if debug : print('input',x.shape)
        y = self.fullyConnected(x)
        if debug : print('output',y.shape)
        return y

### Model analysis

In [ ]:
import torch

if torch.cuda.is_available():
    my_device = torch.device("cuda:0")
    print("Running on CUDA.")
else:
    my_device = torch.device("cpu")
    print("No Cuda Available")

net = ANN( num_classes=3 )

net = net.to(my_device)

a = torch.rand( (1, 4) )
a = a.to(my_device)
b = net( a , debug=True )

del a, b, net

In [ ]:
from torchsummary import summary

net = ANN( num_classes=3 )

net = net.to(my_device)

summary(net, input_size=(120, 4), batch_size=1)

del net

### Training functions

In [ ]:
from sklearn.model_selection import train_test_split
from datetime import datetime

from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
import copy
from tqdm import tqdm

def train ( dataset, prefix=None, upper_bound=101.0, save=False, epochs=100, 
           lr=1e-1, device='cpu', debug=False, layers2tensorboard=True, lambda_reg=0 ) :
    
    num_classes = 3
    
    tensorboard_path = '' # Setar o caminho para salvar o log para o TensorBoard
  
    net = ANN( num_classes )
    net.to(device)

    optimizer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=lambda_reg)
    criterion = nn.CrossEntropyLoss()

    now = datetime.now()
    suffix = now.strftime("%Y%m%d_%H%M%S")
    prefix = suffix if prefix is None else prefix + '-' + suffix  

    writer = SummaryWriter( log_dir=tensorboard_path+prefix )
    
    accuracies = []
    max_accuracy = -1.0  
    
    data_loader = train_test_split( dataset.data,
                                     dataset.target,
                                     test_size=0.2,
                                     random_state=1 )
    
    train_x,test_x,train_label,test_label = data_loader
    
    train_x = torch.from_numpy(train_x).float()
    train_x = train_x.to(device)
    train_label = torch.from_numpy(train_label).float()
    train_label = train_label.to(device)
    
    test_x = torch.from_numpy(test_x).float()
    test_x = test_x.to(device)
    test_label = torch.from_numpy(test_label).float()
    test_label = test_label.to(device)
    
    writer.add_graph(net, train_x)
    
    for epoch in tqdm( range(epochs) , desc='Training epochs...' ) :
        
        # Set Pytorch variables
        net.train()
        optimizer.zero_grad()
                            
        # Forward step
        predict_y = net( train_x )
        
        # Loss
        error = criterion( predict_y , train_label.long() )

        # Back propagation
        error.backward()
        optimizer.step()

        # Accuracies:
        predict_ys = torch.max( predict_y, axis=1 )[1]
        correct    = torch.sum( predict_ys == train_label )
        accuracy_train = correct/train_x.size(0)
        
        accuracy_test = validate(net, test_x, test_label, device=device)
        accuracies.append(accuracy_test)
        
        # Tensor board writing
        writer.add_scalar( 'Loss/train', error.item(), epoch )
        writer.add_scalar( 'Accuracy/train', accuracy_train, epoch )
        writer.add_scalar( 'Accuracy/test', accuracy_test, epoch )

        if layers2tensorboard :
            plot_layers( net, writer, epoch )

        # Test model
        if accuracy_test > max_accuracy:
            best_model = copy.deepcopy(net)
            max_accuracy = accuracy_test
            print(f'Saving the best model at epoch {epoch+1:3d} ' + 
                    f'with Accuracy: {accuracy_test:8.4f}%')
      
        if debug : print( f'Epoch: {epoch+1:3d} |' 
                         + f'Accuracy Test: {accuracy_test:3.4f}%' )

        if accuracy_test > upper_bound :
            break
   
    if save : 
        models_path = '' # Setar o caminho para salvar o modelo final
        path = f'{models_path}ANN-iris-{max_accuracy:.2f}.pkl'
        torch.save( best_model, path )
        print( f'Model saved in: {path}' )
  
    plt.figure(figsize=(16, 8))
    plt.plot(accuracies)

    writer.flush()
    writer.close()

    return best_model

In [ ]:
def validate ( model , test_x, test_label , device='cpu') :

    model.eval()

    correct = 0
    sum = 0

    predict_y = model( test_x ).detach()
    predict_ys = torch.max( predict_y, axis=1 )[1]
    sum = sum + test_x.size(0)
    correct = correct + torch.sum(predict_ys == test_label)
  
    return correct.to('cpu').numpy()*100./sum

In [ ]:
def plot_layers ( net , writer, epoch ) :
    layers = list(net.fullyConnected.modules())

    layer_id = 1
    for layer in layers:
        if isinstance(layer, nn.Linear) :
            writer.add_histogram(f'Bias/linear-{layer_id}', layer.bias, epoch )
            writer.add_histogram(f'Weight/linear-{layer_id}', layer.weight, epoch )
            writer.add_histogram(f'Grad/linear-{layer_id}', layer.weight.grad, epoch )
            layer_id += 1

### Run the training phase

In [ ]:
from sklearn.datasets import load_iris

if torch.cuda.is_available():
    my_device = torch.device("cuda:0")
    print("Running on CUDA.")
else:
    my_device = torch.device("cpu")
    print("No Cuda Available")
    
dataset = 'Iris'
epochs = 500
lr = 1e-1
lambda_reg = 1e-1
prefix = 'ANN-{}-e-{}-lr-{}'.format(dataset, epochs, lr)

iris = load_iris()
dataset = iris

net = train( dataset=dataset, epochs=epochs, device=my_device, 
            upper_bound=101.0, lr=lr, lambda_reg=lambda_reg,
            layers2tensorboard=True, save=False, prefix=prefix
           )